# Import

In [2]:
!pip install wandb
!pip install scipy
!pip install scikit-learn
!pip install tsfel
!pip install datetime
!pip install pandas==1.5.3
!pip install keras-tuner
!pip install fastdtw
!pip install tensorflow-probability

  Using cached fastdtw-0.3.4.tar.gz (133 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512607 sha256=861472c808369c3e24059f77c049e1acacd825996404cea7de5c478880581a0d
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d513c736bb93910277e1528ef4
Successfully built fastdtw


# Run experiments on Colab

In [20]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
drive.flush_and_unmount()
drive.mount('/content/drive')
#save changes on Drive

Mounted at /content/drive


In [1]:
import os
os.chdir('/content/drive/MyDrive/MLinApps-Proj-FP01')

%run main.py --experiment MTGFLOW --cpu


MTGFLOW experiments
--- LOADING TRAINING SET FOR TUNING ---
Loading data.
Found 31 different actions.
Loading data done.

--- TUNING SCALER AND CORR PARAMETERS ---
--- LOADING TRAINING SET ---
Loading data.
Found 31 different actions.
Loading data done.

--- PREPROCESSING TRAINING SPLIT 0 WITH OVERLAP 0.7 ---
Loading data.
Found 31 different actions.
Loading data done.

--- PREPROCESSING TRAINING SPLIT 1 WITH OVERLAP 0.7 ---
Loading data.
Found 31 different actions.
Loading data done.

--- PREPROCESSING TRAINING SPLIT 2 WITH OVERLAP 0.7 ---
Loading data.
Found 31 different actions.
Loading data done.

--- PREPROCESSING TRAINING SPLIT 3 WITH OVERLAP 0.7 ---
window split train size:
(3183, 100, 51)
window action train size:
3183
--- LOADING TEST SET ---
Loading data.
Found 31 different actions.
Loading data done.

--- PREPROCESSING TEST SPLIT 0 ---
Loading data.
Found 31 different actions.
Loading data done.

--- PREPROCESSING TEST SPLIT 1 ---
window split test size:
(34077, 100, 51)
win

wandb: Currently logged in as: fede-bussolino (fede-bussolino-Politecnico di Torino). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


--------------------------START TRAINING--------------------------
[5e-05, 4.5e-05, 4.05e-05, 3.6450000000000005e-05, 3.2805e-05, 2.9524500000000005e-05]
epoch: 0/50
Epoch 1 train loss: 1.9322, lr: 4.999999873689376e-05
saved
epoch: 1/50


KeyboardInterrupt: 

### publish by download and run on vscode

# Random test

In [ ]:

import os
import time
import tsfel
import warnings
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
import tensorflow as tf
import keras_tuner
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

ROOT = '/content/drive/MyDrive/MLinApps-Proj-FP01'

ROOTDIR_DATASET_NORMAL = ROOT + "/normal/"
ROOTDIR_DATASET_ANOMALY = ROOT + "/collisions/"
filepath_csv = [ROOTDIR_DATASET_NORMAL + f"rec{r}_20220811_rbtc_0.1s.csv" for r in [0, 2, 3, 4]]
filepath_meta = [ROOTDIR_DATASET_NORMAL + f"rec{r}_20220811_rbtc_0.1s.metadata" for r in [0, 2, 3, 4]]
filepath_csv_anomaly = [ROOTDIR_DATASET_ANOMALY + f"rec{r}_collision_20220811_rbtc_0.1s.csv" for r in [1, 5]]
filepath_meta_anomaly = [ROOTDIR_DATASET_ANOMALY + f"rec{r}_collision_20220811_rbtc_0.1s.metadata" for r in [1, 5]]


def get_df_action(filepaths_csv, filepaths_meta, action2int=None, delimiter=";"):
    # Load dataframes
    #print("Loading data.")
    # Make dataframes
    # Some classes show the output boolean parameter as True rather than true. Fix here
    dfs_meta = list()
    for filepath in filepaths_meta:
        print(f"loading from: {filepath}")
        df_m = pd.read_csv(filepath, sep=delimiter)
        df_m.str_repr = df_m.str_repr.str.replace('True', 'true')
        df_m['filepath'] = filepath
        dfs_meta.append(df_m)


    df_meta = pd.concat(dfs_meta)
    df_meta.index = pd.to_datetime(df_meta.init_timestamp.astype('datetime64[ms]'), format="%Y-%m-%dT%H:%M:%S.%f")
    df_meta['completed_timestamp'] = pd.to_datetime(df_meta.completed_timestamp.astype('datetime64[ms]'),
                                                    format="%Y-%m-%dT%H:%M:%S.%f")
    df_meta['init_timestamp'] = pd.to_datetime(df_meta.init_timestamp.astype('datetime64[ms]'),
                                               format="%Y-%m-%dT%H:%M:%S.%f")

    # Eventually reduce number of classes
    # df_meta['str_repr'] = df_meta.str_repr.str.split('=', expand = True,n=1)[0]
    # df_meta['str_repr'] = df_meta.str_repr.str.split('(', expand=True, n=1)[0]

    actions = df_meta.str_repr.unique()
    dfs = [pd.read_csv(filepath_csv, sep=";") for filepath_csv in filepaths_csv]
    df = pd.concat(dfs)

    # Sort columns by name !!!
    df = df.sort_index(axis=1)

    # Set timestamp as index
    df.index = pd.to_datetime(df.time.astype('datetime64[ms]'), format="%Y-%m-%dT%H:%M:%S.%f")
    # Drop useless columns
    columns_to_drop = [column for column in df.columns if "Abb" in column or "Temperature" in column]
    df.drop(["machine_nameKuka Robot_export_active_energy",
             "machine_nameKuka Robot_import_reactive_energy"] + columns_to_drop, axis=1, inplace=True)
    signals = df.columns

    df_action = list()
    for action in actions:
        for index, row in df_meta[df_meta.str_repr == action].iterrows():
            start = row['init_timestamp']
            end = row['completed_timestamp']
            df_tmp = df.loc[start: end].copy()
            df_tmp['action'] = action
            # Duration as string (so is not considered a feature)
            df_tmp['duration'] = str((row['completed_timestamp'] - row['init_timestamp']).total_seconds())
            df_action.append(df_tmp)
    df_action = pd.concat(df_action, ignore_index=True)
    df_action.index = pd.to_datetime(df_action.time.astype('datetime64[ms]'), format="%Y-%m-%dT%H:%M:%S.%f")
    df_action = df_action[~df_action.index.duplicated(keep='first')]

    # Drop NaN
    df = df.dropna(axis=0)
    df_action = df_action.dropna(axis=0)

    if action2int is None:
        action2int = dict()
        j = 1
        for label in df_action.action.unique():
            action2int[label] = j
            j += 1

    df_merged = df.merge(df_action[['action']], left_index=True, right_index=True, how="left")
    # print(f"df_merged len: {len(df_merged)}")
    # Where df_merged in NaN Kuka is in idle state
    df_idle = df_merged[df_merged['action'].isna()].copy()
    df_idle['action'] = 'idle'
    df_idle['duration'] = df_action.duration.values.astype(float).mean().astype(str)
    df_action = pd.concat([df_action, df_idle])

    df_action = df_action.sort_index()

    # ile label must be 0 for debug mode
    action2int['idle'] = 0
    #print(f"Found {len(set(df_action['action']))} different actions.")
    #print("Loading data done.\n")

    return df_action, df, df_meta, action2int




def sample_as_splitter(signal, window_size, overlap=0):
    """sample same size as splitter of tsfel

    Parameters
    ----------
    signal : nd-array or pandas DataFrame
        input signal
    window_size : int
        number of points of window size
    overlap : float
        percentage of overlap, value between 0 and 1 (exclusive)
        Default: 0
    Returns
    -------
    list
        list of signal windows
    """
    if not isinstance(window_size, int):
        raise SystemExit("window_size must be an integer.")
    step = int(round(window_size)) if overlap == 0 else int(round(window_size * (1 - overlap)))
    if step == 0:
        raise SystemExit(
            "Invalid overlap. " "Choose a lower overlap value.",
        )
    if len(signal) % window_size == 0 and overlap == 0:
        return [signal[i] for i in range(0, len(signal), step)]
    else:
        return [signal[i] for i in range(0, len(signal) - window_size + 1, step)]


def MTGFLOWLOADER_action(opt):

    #Use same overlap for test and train set

    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=UserWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)

    if opt['num_features']==50:
        mode = 0
    elif opt['num_features']==51:
        mode = 1
    else:
        assert False, "not implemented"

    frequency = 1/10
    duration = opt['window_size']*frequency
    overlap = (opt['window_size'] - opt['stride']) / opt['window_size']


    print("--- LOADING TRAINING SET FOR TUNING ---")
    #NOTE: all action2int unique values must appear in training set
    #action2int and action_cols must not be changed at test time
    X_train_action, _, _, action2int = get_df_action(filepath_csv, filepath_meta)

    scaler = StandardScaler()

    shape = (-1, opt['window_size'], opt['num_features'])

    print("--- TUNING SCALER AND CORR PARAMETERS ---")

    #int encoding
    X_train_action["action"] = X_train_action["action"].map(action2int)

    X_train = X_train_action.fillna(0) #1 fill null values

    if mode != 1:
        X_train = X_train.drop(["time", "duration", "action"], axis=1) #2 drop timestamps columns
    else:
        X_train = X_train.drop(["time", "duration"], axis=1) #2 drop timestamps columns

    corr_features = tsfel.correlated_features(X_train,threshold=0.95) #action should not be too much correlated

    X_train = X_train.drop(corr_features, axis=1) #3 drop corr features

    scaler.fit_transform(X_train)

    print("--- LOADING TRAINING SET ---")

    window_splits_train = []
    window_action_train = []

    for i in range(4):
        #get one csv file at a time, discard incorrect (cross-file) time windows
        #--> start from first of at least window_size size)

        X_train_action, _, _, _ = get_df_action([filepath_csv[i],], [filepath_meta[i],])

        print(f"--- PREPROCESSING TRAINING SPLIT {i} WITH OVERLAP {overlap} ---")
        #actual preprocessing

        #int encoding
        X_train_action["action"] = X_train_action["action"].map(action2int)

        window_action_train.extend(sample_as_splitter(X_train_action["action"],opt["window_size"], overlap=overlap)) #need ations mapping to rescale by action


        X_train = X_train_action.fillna(0) #1 fill null values

        if mode != 1:
            X_train = X_train.drop(["time","duration","action"], axis=1) #2 drop timestamps columns
        else:
            X_train = X_train.drop(["time","duration"], axis=1) #2 drop timestamps columns

        X_train = X_train.drop(corr_features, axis=1) #3 drop corr features
        X_train = scaler.transform(X_train) #4 normalize


        window_splits_train.extend(tsfel.utils.signal_processing.signal_window_splitter(X_train.copy(), opt['window_size'], overlap=overlap))


    #output of scaler is np array
    window_splits_train = np.concatenate(window_splits_train, axis=0)
    window_splits_train = window_splits_train.reshape(shape)

    overlap = (opt['window_size'] - 1) / opt['window_size']
    #overlap = 0

    print("window split train size:")
    print(window_splits_train.shape)
    print("window action train size:")
    print(len(window_action_train))
    assert len(window_action_train) == len(window_splits_train) , "error loading train set actions"

    print("--- LOADING TEST SET ---")

    window_splits_test = []
    window_action_test = []
    window_test_times = pd.DataFrame()

    for i in range(2):
        #get one csv file at a time, discard incorrect (cross-file) time windows
        #--> start from first of at least window_size size)
        X_test_action, _, _, _ = get_df_action([filepath_csv_anomaly[i],], [filepath_meta_anomaly[i],])

        print(f"--- PREPROCESSING TEST SPLIT {i} ---")

        df_time = pd.DataFrame()
        time = pd.to_datetime(X_test_action.time.astype('datetime64[ms]'), format="%Y-%m-%dT%H:%M:%S.%f")
        #NOTE: X_test_action returns always timestamp of first file

        df_time['start'] = sample_as_splitter(time, opt["window_size"], overlap=overlap)
        df_time['end'] = df_time['start'] + pd.to_timedelta(duration, 's')

        #int encoding
        X_test_action["action"] = X_test_action["action"].map(action2int)
        window_action_test.extend(sample_as_splitter(X_test_action["action"],opt["window_size"], overlap=overlap)) #need ations mapping to rescale by action

        X_test = X_test_action.fillna(0) #1 fill null values
        if mode != 1:
            X_test = X_test.drop(["time","duration","action"], axis=1)  #2 drop timestamps columns
        else:
            X_test = X_test.drop(["time","duration"], axis=1)

        X_test = X_test.drop(corr_features, axis=1) #3 drop corr features
        X_test = scaler.transform(X_test) #4 normalize

        window_splits_test.extend(tsfel.utils.signal_processing.signal_window_splitter(X_test.copy(), opt['window_size'], overlap=overlap))
        #append new timestamps
        window_test_times = pd.concat([window_test_times, df_time], axis=0)

    #output of scaler is np array
    window_splits_test = np.concatenate(window_splits_test, axis=0)
    window_splits_test = window_splits_test.reshape(shape)

    print("window split test size:")
    print(window_splits_test.shape)
    print("window action test size:")
    print(len(window_action_test))
    print("timestamps")
    print(len(window_test_times))

    assert len(window_test_times) == len(window_splits_test), "time and data mismatch"
    assert len(window_action_test) == len(window_splits_test) , "error loading test set actions"

    return window_splits_train, window_action_train, window_splits_test, window_test_times, window_action_test

opt = {'num_features':50, 'window_size':100, 'stride':100}

_, _, _, test_times, _ = MTGFLOWLOADER_action(opt)

--- LOADING TRAINING SET FOR TUNING ---
loading from: /content/drive/MyDrive/local_proj_new/normal/rec0_20220811_rbtc_0.1s.metadata
loading from: /content/drive/MyDrive/local_proj_new/normal/rec2_20220811_rbtc_0.1s.metadata
loading from: /content/drive/MyDrive/local_proj_new/normal/rec3_20220811_rbtc_0.1s.metadata
loading from: /content/drive/MyDrive/local_proj_new/normal/rec4_20220811_rbtc_0.1s.metadata
--- TUNING SCALER AND CORR PARAMETERS ---
--- LOADING TRAINING SET ---
loading from: /content/drive/MyDrive/local_proj_new/normal/rec0_20220811_rbtc_0.1s.metadata
--- PREPROCESSING TRAINING SPLIT 0 WITH OVERLAP 0.0 ---
loading from: /content/drive/MyDrive/local_proj_new/normal/rec2_20220811_rbtc_0.1s.metadata
--- PREPROCESSING TRAINING SPLIT 1 WITH OVERLAP 0.0 ---
loading from: /content/drive/MyDrive/local_proj_new/normal/rec3_20220811_rbtc_0.1s.metadata
--- PREPROCESSING TRAINING SPLIT 2 WITH OVERLAP 0.0 ---
loading from: /content/drive/MyDrive/local_proj_new/normal/rec4_20220811_rbtc

In [ ]:
import numpy as np
import pandas as pd

ROOTDIR_DATASET_ANOMALY = ROOT + "/collisions/"
filepath_csv_anomaly = [ROOTDIR_DATASET_ANOMALY + f"rec{r}_collision_20220811_rbtc_0.1s.csv" for r in [1, 5]]
filepath_meta_anomaly = [ROOTDIR_DATASET_ANOMALY + f"rec{r}_collision_20220811_rbtc_0.1s.metadata" for r in [1, 5]]


def calculate_auc(x, y):
    x=np.array(x)
    y=np.array(y)
    # Ensure TPR and FPR arrays are sorted together by FPR (ascending)
    sorted_idx = np.argsort(x)
    x = x[sorted_idx]
    y = y[sorted_idx]

    # Calculate trapezoidal areas for each interval
    auc = np.trapz(y, x)

    return auc


def convert(times_windowed):

    df = pd.DataFrame(columns = ['start','end'])
    for time_window in times_windowed: #iterate over dimension 0, extract window timestamps
        start = time_window[0][0]
        end = time_window[-1][0]
        df = pd.concat([df, pd.DataFrame({'start':start,'end':end})], ignore_index=True)

    return df


def compute_average_score(scores, opt):

    window_size = opt['window_size']

    adjusted_anomalies = np.zeros(len(scores))

    for i in range(len(scores)-window_size):
        adjusted_anomalies[i:i + window_size] += scores[i]

    counts = np.zeros(len(scores))
    for i in range(len(scores)-window_size):
        counts[i:i + window_size] += 1

    adjusted_anomalies = adjusted_anomalies / counts

    return adjusted_anomalies


def metrics_by_point_vectorized(scores, time_collision, full=False, convert=False):
    #score is attributed to last point in window

    if full:
        thresholds = np.sort(scores)
    else:
        thresholds = np.linspace(scores.min(), scores.max(), num=300)

    if convert:
        time_collision = convert(time_collision) #TADGANLOADER conversion

    collisions = pd.read_excel(ROOT +"/collisions/20220811_collisions_timestamp.xlsx")
    collisions_init = collisions[collisions['Inizio/fine'] == "i"].Timestamp - pd.to_timedelta([2] * len(collisions[collisions['Inizio/fine'] == "i"].Timestamp), 'h')
    collision_end = collisions[collisions['Inizio/fine'] == "f"].Timestamp - pd.to_timedelta([2] * len(collisions[collisions['Inizio/fine'] == "f"].Timestamp), 'h')

    time_collision = time_collision[:len(scores)]
    assert len(scores) == len(time_collision), "unmatching score/thresholds/timestamp"
    print(f"--- LOADED {len(collisions_init)} COLLISIONS ---")

    # Convert timestamps to numpy arrays
    start_times = time_collision['start'].to_numpy().astype('datetime64[ns]')
    end_times = time_collision['end'].to_numpy().astype('datetime64[ns]')

    collisions_init_np = collisions_init.to_numpy().astype('datetime64[ns]')
    collisions_end_np = collision_end.to_numpy().astype('datetime64[ns]')

    # Create a mask for each threshold
    threshold_masks = scores[:, np.newaxis] >= thresholds

    n_samples = len(scores)

    # Calculate metrics for each threshold
    results = []
    for threshold_mask in threshold_masks.T:

        pos_pred  = np.sum(threshold_mask)
        neg_pred = n_samples - pos_pred

        # count anomaly timestamps included in an anomaly window -> tp
        collision_in_window = ((start_times[threshold_mask] >= collisions_init_np[:, np.newaxis]) & \
                              (start_times[threshold_mask] < collisions_end_np[:, np.newaxis]))

        tp = np.sum(collision_in_window) #overall sum is necessary
        fp = pos_pred - tp

        not_threshold_mask = np.where(threshold_mask, False, True)

        # count non anomaly timestamps included in an anomaly window -> fn
        false_not_collision_in_window = ((start_times[not_threshold_mask] >= collisions_init_np[:, np.newaxis]) & \
                                        (start_times[not_threshold_mask] < collisions_end_np[:, np.newaxis]))

        fn = np.sum(false_not_collision_in_window)
        tn = neg_pred - fn

        anomaly_indices = np.where(threshold_mask)[0][np.any(collision_in_window, axis=0)]

        cm_anomaly = np.array([[tn, fp], [fn, tp]])

        precision = tp / (tp + fp) if tp + fp != 0 else 0
        recall = tp / (tp + fn) if tp + fn != 0 else 0
        fpr = fp / (fp + tn) if fp + tn != 0 else 0
        accuracy = (tp + tn) / (tp + tn + fp + fn) if tp + tn + fp + fn != 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else 0

        results.append((recall, precision, fpr, accuracy, f1, cm_anomaly, anomaly_indices))

    recalls, precisions, fprs, accuracies, f1s, cms, anomaly_indices_list = zip(*results)

    return recalls, precisions, fprs, accuracies, f1s, cms, anomaly_indices_list


f1_res = []
auroc_res = []
auprc_res = []

for i in range(20):
    print(f"test {i}")
    scores = np.random.rand(34077)

    recalls, precisions, fprs, _, f1s, _, _ = metrics_by_point_vectorized(scores, test_times, full=True)
    f1_res.append(max(f1s))
    auroc_res.append(calculate_auc(fprs, recalls))
    auprc_res.append(calculate_auc(recalls, precisions))

f1 = np.array(f1_res)
auroc = np.array(auroc_res)
auprc = np.array(auprc_res)

print(f"f1 mean: {np.mean(f1)}, f1 std: {np.std(f1)}")
print(f"auroc: {np.mean(auroc)}, auroc std: {np.std(auroc)}")
print(f"auprc: {np.mean(auprc)}, auprc std: {np.std(auprc)}")




test 0
--- LOADED 107 COLLISIONS ---
test 1
--- LOADED 107 COLLISIONS ---
test 2
--- LOADED 107 COLLISIONS ---
test 3
--- LOADED 107 COLLISIONS ---
test 4
--- LOADED 107 COLLISIONS ---
test 5
--- LOADED 107 COLLISIONS ---
test 6
--- LOADED 107 COLLISIONS ---
test 7
--- LOADED 107 COLLISIONS ---
test 8
--- LOADED 107 COLLISIONS ---
test 9
--- LOADED 107 COLLISIONS ---
test 10
--- LOADED 107 COLLISIONS ---
test 11
--- LOADED 107 COLLISIONS ---
test 12
--- LOADED 107 COLLISIONS ---
test 13
--- LOADED 107 COLLISIONS ---
test 14
--- LOADED 107 COLLISIONS ---
test 15
--- LOADED 107 COLLISIONS ---
test 16
--- LOADED 107 COLLISIONS ---
test 17
--- LOADED 107 COLLISIONS ---
test 18
--- LOADED 107 COLLISIONS ---
test 19
--- LOADED 107 COLLISIONS ---
f1 mean: 0.21659268377381022, f1 std: 0.0002841619850897175
auroc: 0.500171983961916, auroc std: 0.002892736446929721
auprc: 0.12115531248081213, auprc std: 0.0014416435919406036
